### Week 3
### DSC 550
### Abed Tabbalat

#### Part 1

In [115]:
import pandas as pd
from textblob import TextBlob
from sklearn.metrics import accuracy_score
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\abedt\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abedt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abedt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [116]:
# Import the movie review data as a data frame and ensure that the data is loaded properly.
df = pd.read_csv('labeledTrainData.tsv', sep='\t')
df

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...


In [117]:
# How many of each positive and negative reviews are there?
# will do a groupby() and count()
df_sent = df.groupby('sentiment').count()
df_sent

# answer is a 50 50

,id,review
sentiment,,
0,12500,12500
1,12500,12500


In [118]:
# Adding functions to breakout Subjectivity and Polarity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [119]:
# Use TextBlob to classify each movie review as positive or negative. Assume that a polarity score greater than or equal to zero is a positive sentiment and less than 0 is a negative sentiment.
df['BlobSubjectivity'] = df['review'].apply(getSubjectivity)
df['BlobPolarity'] = df['review'].apply(getPolarity)
df

,id,sentiment,review,BlobSubjectivity,BlobPolarity
0,5814_8,1,With all this stuff going down at the moment w...,0.606746,0.001277
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",0.531111,0.256349
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,0.562933,-0.053941
3,3630_4,0,It must be assumed that those who praised this...,0.492901,0.134753
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,0.459818,-0.024842
...,...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,0.542857,0.102083
24996,5064_1,0,I don't believe they made this film. Completel...,0.462371,0.090813
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...",0.484103,0.145256
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,0.504514,0.065625


In [120]:
# Adding another function to convert polarity to a meaningful answer
def getAnalysis(score):
    if score < 0:
        return 0
    else:
        return 1

In [121]:
# Creating final column to give out the analysis
df['Blob_Sentiment'] = df['BlobPolarity'].apply(getAnalysis)
df

,id,sentiment,review,BlobSubjectivity,BlobPolarity,Blob_Sentiment
0,5814_8,1,With all this stuff going down at the moment w...,0.606746,0.001277,1
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",0.531111,0.256349,1
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,0.562933,-0.053941,0
3,3630_4,0,It must be assumed that those who praised this...,0.492901,0.134753,1
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,0.459818,-0.024842,0
...,...,...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,0.542857,0.102083,1
24996,5064_1,0,I don't believe they made this film. Completel...,0.462371,0.090813,1
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...",0.484103,0.145256,1
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,0.504514,0.065625,1


In [122]:
# Check the accuracy of the Blob model.
blob_accuracy = accuracy_score(df['sentiment'], df['Blob_Sentiment'])
blob_accuracy

0.68524

In [123]:
# For up to five points extra credit, use another prebuilt text sentiment analyzer, e.g., VADER, and repeat steps (3) and (4).
df['Vader_Polarity'] = df['review'].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x))
df['Vader_Sentiment'] = df['Vader_Polarity'].apply(lambda x: int(x['compound'] >= 0))
df

,id,sentiment,review,BlobSubjectivity,BlobPolarity,Blob_Sentiment,Vader_Polarity,Vader_Sentiment
0,5814_8,1,With all this stuff going down at the moment w...,0.606746,0.001277,1,"{'neg': 0.13, 'neu': 0.744, 'pos': 0.126, 'com...",0
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",0.531111,0.256349,1,"{'neg': 0.047, 'neu': 0.739, 'pos': 0.214, 'co...",1
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,0.562933,-0.053941,0,"{'neg': 0.142, 'neu': 0.8, 'pos': 0.058, 'comp...",0
3,3630_4,0,It must be assumed that those who praised this...,0.492901,0.134753,1,"{'neg': 0.066, 'neu': 0.878, 'pos': 0.056, 'co...",0
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,0.459818,-0.024842,0,"{'neg': 0.119, 'neu': 0.741, 'pos': 0.14, 'com...",1
...,...,...,...,...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,0.542857,0.102083,1,"{'neg': 0.026, 'neu': 0.822, 'pos': 0.152, 'co...",1
24996,5064_1,0,I don't believe they made this film. Completel...,0.462371,0.090813,1,"{'neg': 0.087, 'neu': 0.66, 'pos': 0.253, 'com...",1
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...",0.484103,0.145256,1,"{'neg': 0.055, 'neu': 0.789, 'pos': 0.156, 'co...",1
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,0.504514,0.065625,1,"{'neg': 0.136, 'neu': 0.768, 'pos': 0.096, 'co...",0


In [124]:
# Check the accuracy of the Vader model.
vader_accuracy = accuracy_score(df['sentiment'], df['Vader_Sentiment'])
vader_accuracy

0.69356

In [125]:
# Comparing Blob model to Vader model
print(f'Blob Model Accuracy: {blob_accuracy}')
print(f'Vader Model Accuracy: {vader_accuracy}')

Blob Model Accuracy: 0.68524
Vader Model Accuracy: 0.69356


#### Part 2

In [126]:
# Convert all text to lowercase letters.
df2 = df[['id', 'sentiment', 'review']]
df2['review_lower'] = df2['review'].str.lower()
df2

<ipython-input-126-d8dcc207245d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['review_lower'] = df2['review'].str.lower()


,id,sentiment,review,review_lower
0,5814_8,1,With all this stuff going down at the moment w...,with all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","\the classic war of the worlds\"" by timothy hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,the film starts with a manager (nicholas bell)...
3,3630_4,0,It must be assumed that those who praised this...,it must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...
...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,it seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...,i don't believe they made this film. completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...","guy is a loser. can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,this 30 minute documentary buñuel made in the ...


In [127]:
# Remove punctuation and special characters from the text.
df2['review_nopunc'] = df2['review_lower'].str.replace(r'[^\w\s]+', '')
df2

<ipython-input-127-8c93bd40aeb4>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['review_nopunc'] = df2['review_lower'].str.replace(r'[^\w\s]+', '')
<ipython-input-127-8c93bd40aeb4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['review_nopunc'] = df2['review_lower'].str.replace(r'[^\w\s]+', '')


,id,sentiment,review,review_lower,review_nopunc
0,5814_8,1,With all this stuff going down at the moment w...,with all this stuff going down at the moment w...,with all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","\the classic war of the worlds\"" by timothy hi...",the classic war of the worlds by timothy hines...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,the film starts with a manager (nicholas bell)...,the film starts with a manager nicholas bell g...
3,3630_4,0,It must be assumed that those who praised this...,it must be assumed that those who praised this...,it must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...
...,...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,it seems like more consideration has gone into...,it seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...,i don't believe they made this film. completel...,i dont believe they made this film completely ...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...","guy is a loser. can't get girls, needs to buil...",guy is a loser cant get girls needs to build u...
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,this 30 minute documentary buñuel made in the ...,this 30 minute documentary buñuel made in the ...


In [128]:
# Creating function to remove stop workds and tokenize
def token_nostop(tweet):
    stop_words = stopwords.words('english')
    tweet_token = word_tokenize(tweet)
    tweet_stopword = [x for x in tweet_token if x not in stop_words]
    return tweet_stopword

In [129]:
# Remove stop words.
stop_words = stopwords.words('english')
df2['review_token'] = df2['review_nopunc'].apply(token_nostop)
df2

<ipython-input-129-f7e386b686f8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['review_token'] = df2['review_nopunc'].apply(token_nostop)


,id,sentiment,review,review_lower,review_nopunc,review_token
0,5814_8,1,With all this stuff going down at the moment w...,with all this stuff going down at the moment w...,with all this stuff going down at the moment w...,"[stuff, going, moment, mj, ive, started, liste..."
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","\the classic war of the worlds\"" by timothy hi...",the classic war of the worlds by timothy hines...,"[classic, war, worlds, timothy, hines, enterta..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,the film starts with a manager (nicholas bell)...,the film starts with a manager nicholas bell g...,"[film, starts, manager, nicholas, bell, giving..."
3,3630_4,0,It must be assumed that those who praised this...,it must be assumed that those who praised this...,it must be assumed that those who praised this...,"[must, assumed, praised, film, greatest, filme..."
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...,"[superbly, trashy, wondrously, unpretentious, ..."
...,...,...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,it seems like more consideration has gone into...,it seems like more consideration has gone into...,"[seems, like, consideration, gone, imdb, revie..."
24996,5064_1,0,I don't believe they made this film. Completel...,i don't believe they made this film. completel...,i dont believe they made this film completely ...,"[dont, believe, made, film, completely, unnece..."
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...","guy is a loser. can't get girls, needs to buil...",guy is a loser cant get girls needs to build u...,"[guy, loser, cant, get, girls, needs, build, p..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,this 30 minute documentary buñuel made in the ...,this 30 minute documentary buñuel made in the ...,"[30, minute, documentary, buñuel, made, early,..."


In [130]:
# Creating function to apply stemming
def stemming(tweet):
    pstemmer = PorterStemmer()
    stem_ret = [pstemmer.stem(x) for x in tweet]
    return stem_ret

In [131]:
# Apply NLTK’s PorterStemmer.
df2['review_stem'] = df2['review_token'].apply(stemming)
df2

<ipython-input-131-b591cf04e160>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['review_stem'] = df2['review_token'].apply(stemming)


,id,sentiment,review,review_lower,review_nopunc,review_token,review_stem
0,5814_8,1,With all this stuff going down at the moment w...,with all this stuff going down at the moment w...,with all this stuff going down at the moment w...,"[stuff, going, moment, mj, ive, started, liste...","[stuff, go, moment, mj, ive, start, listen, mu..."
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","\the classic war of the worlds\"" by timothy hi...",the classic war of the worlds by timothy hines...,"[classic, war, worlds, timothy, hines, enterta...","[classic, war, world, timothi, hine, entertain..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,the film starts with a manager (nicholas bell)...,the film starts with a manager nicholas bell g...,"[film, starts, manager, nicholas, bell, giving...","[film, start, manag, nichola, bell, give, welc..."
3,3630_4,0,It must be assumed that those who praised this...,it must be assumed that those who praised this...,it must be assumed that those who praised this...,"[must, assumed, praised, film, greatest, filme...","[must, assum, prais, film, greatest, film, ope..."
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...,"[superbly, trashy, wondrously, unpretentious, ...","[superbl, trashi, wondrous, unpretenti, 80, ex..."
...,...,...,...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,it seems like more consideration has gone into...,it seems like more consideration has gone into...,"[seems, like, consideration, gone, imdb, revie...","[seem, like, consider, gone, imdb, review, fil..."
24996,5064_1,0,I don't believe they made this film. Completel...,i don't believe they made this film. completel...,i dont believe they made this film completely ...,"[dont, believe, made, film, completely, unnece...","[dont, believ, made, film, complet, unnecessar..."
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...","guy is a loser. can't get girls, needs to buil...",guy is a loser cant get girls needs to build u...,"[guy, loser, cant, get, girls, needs, build, p...","[guy, loser, cant, get, girl, need, build, pic..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,this 30 minute documentary buñuel made in the ...,this 30 minute documentary buñuel made in the ...,"[30, minute, documentary, buñuel, made, early,...","[30, minut, documentari, buñuel, made, earli, ..."


In [132]:
# creating the final view after untokenizing the words
df2['review_end'] = df2['review_stem'].apply(lambda x: ' '.join(x))
df2

,id,sentiment,review,review_lower,review_nopunc,review_token,review_stem,review_end
0,5814_8,1,With all this stuff going down at the moment w...,with all this stuff going down at the moment w...,with all this stuff going down at the moment w...,"[stuff, going, moment, mj, ive, started, liste...","[stuff, go, moment, mj, ive, start, listen, mu...",stuff go moment mj ive start listen music watc...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","\the classic war of the worlds\"" by timothy hi...",the classic war of the worlds by timothy hines...,"[classic, war, worlds, timothy, hines, enterta...","[classic, war, world, timothi, hine, entertain...",classic war world timothi hine entertain film ...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,the film starts with a manager (nicholas bell)...,the film starts with a manager nicholas bell g...,"[film, starts, manager, nicholas, bell, giving...","[film, start, manag, nichola, bell, give, welc...",film start manag nichola bell give welcom inve...
3,3630_4,0,It must be assumed that those who praised this...,it must be assumed that those who praised this...,it must be assumed that those who praised this...,"[must, assumed, praised, film, greatest, filme...","[must, assum, prais, film, greatest, film, ope...",must assum prais film greatest film opera ever...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...,"[superbly, trashy, wondrously, unpretentious, ...","[superbl, trashi, wondrous, unpretenti, 80, ex...",superbl trashi wondrous unpretenti 80 exploit ...
...,...,...,...,...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,it seems like more consideration has gone into...,it seems like more consideration has gone into...,"[seems, like, consideration, gone, imdb, revie...","[seem, like, consider, gone, imdb, review, fil...",seem like consider gone imdb review film went ...
24996,5064_1,0,I don't believe they made this film. Completel...,i don't believe they made this film. completel...,i dont believe they made this film completely ...,"[dont, believe, made, film, completely, unnece...","[dont, believ, made, film, complet, unnecessar...",dont believ made film complet unnecessari firs...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...","guy is a loser. can't get girls, needs to buil...",guy is a loser cant get girls needs to build u...,"[guy, loser, cant, get, girls, needs, build, p...","[guy, loser, cant, get, girl, need, build, pic...",guy loser cant get girl need build pick strong...
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,this 30 minute documentary buñuel made in the ...,this 30 minute documentary buñuel made in the ...,"[30, minute, documentary, buñuel, made, early,...","[30, minut, documentari, buñuel, made, earli, ...",30 minut documentari buñuel made earli 1930 on...


In [133]:
# Forming a matrix of the words in review_end column
words_amt = CountVectorizer().fit_transform(df2['review_end'])
words_amt.shape

(25000, 92528)

In [134]:
# Applying tfidf vectorizer function to the matrix
matrix_vec = TfidfVectorizer().fit_transform(df2['review_end'])
matrix_vec.shape

(25000, 92528)